## Convert behavior data

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('paper')
sns.set_style('ticks')

/usr/local/python-2.7.6/lib/python2.7/site-packages/matplotlib-1.5.1-py2.7-linux-x86_64.egg/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/python-2.7.6/lib/python2.7/site-packages/matplotlib-1.5.1-py2.7-linux-x86_64.egg/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
import numpy as np
import pandas as pd
from glob import glob
from os.path import join, exists
from shutil import copytree
import json

###Set directory and session information

In [3]:
directory = '/tier2/freeman/Nick/lfov.calibration'

In [4]:
key = '2016-04-01'
name = 'anm-0326049'

Check if behavior data exists

In [5]:
path = join(directory, 'unprocessed', 'behavior', key, name)
print exists(path)

True


###Load trial data

In [6]:
files = glob(join(path, 'trial-*.csv'))
files.sort()
trials = [pd.read_csv(x) for x in files]

Create session table

In [7]:
session = pd.concat(trials)
session.reset_index(inplace=True)
session.rename(columns={'index': 'tick'}, inplace=True)

Add global time

In [8]:
trial = session.groupby(['number']).last()
offsets = np.concatenate(([0],trial.time.values))
offsets = np.cumsum(offsets)
session['allTime'] = session.time + offsets[session.number]

Add filtered speed

In [9]:
speed = (session.xSpeed.pow(2)+session.ySpeed.pow(2)).pow(.5)
session['speed'] = np.convolve(speed,np.ones(50)/50, mode='same')    

### Save full-length session

In [10]:
session.to_csv(join(path, 'session.csv'))

###Copy data to new directory

In [11]:
savepath = join(directory, 'reprocessed', name, key, 'behavior')
copytree(path,savepath)